# Imports

In [ ]:
from numpy.random import seed
seed(888)

In [ ]:
import os
import numpy as np
import tensorflow as tf

from time import strftime
from PIL import Image

# Constants

In [ ]:
X_TRAIN_PATH = 'Handwriting/digit_xtrain.csv'
X_TEST_PATH = 'Handwriting/digit_xtest.csv'
Y_TRAIN_PATH = 'Handwriting/digit_ytrain.csv'
Y_TEST_PATH = 'Handwriting/digit_ytest.csv'

LOGGING_PATH = 'tensorboard_Handwriting_logs/'

NR_CLASSES = 10

VALIDATION_SIZE = 10000

IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1

TOTAL_INPUTS = IMAGE_WIDTH * IMAGE_HEIGHT * CHANNELS

# Get the Data

In [ ]:
%%time

y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter = ',', dtype = int)
y_test = np.loadtxt(Y_TEST_PATH, delimiter = ',', dtype = int)
x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter = ',', dtype = int)
x_test = np.loadtxt(X_TEST_PATH, delimiter = ',', dtype = int)

CPU times: user 59.4 s, sys: 2.69 s, total: 1min 2s
Wall time: 1min 6s


# Explore

In [ ]:
x_train_all.shape

(60000, 784)

In [ ]:
x_train_all[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

# Proprocessing

In [ ]:
x_train_all, x_test = x_train_all / 255.0, x_test / 255.0

### Hot encoding

In [ ]:
y_train_all = np.eye(NR_CLASSES)[y_train_all]

In [ ]:
y_test = np.eye(NR_CLASSES)[y_test]

### Create validation dataset from training data

In [ ]:
x_validation = x_train_all[:VALIDATION_SIZE]
y_validation = y_train_all[:VALIDATION_SIZE]

In [ ]:
x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

# Setup Tensorflow Graph

In [ ]:
x = tf.placeholder(tf.float32, shape = [None, 784], name= 'X')
y = tf.placeholder(tf.float32, shape = [None, NR_CLASSES], name = 'labels')

## Neural Network Architecture

### Hyperparameters

In [ ]:
nr_epochs = 10
learning_rate = 0.001

n_hidden1 = 512
n_hidden2 = 64

### Layer 1

In [ ]:
# with tf.name_scope('first_layer'):

#     initial_w1 = tf.truncated_normal(shape = [TOTAL_INPUTS, n_hidden1], stddev = 0.1, seed = 42)
#     w1 = tf.Variable(initial_value = initial_w1, name = 'w1')

#     initial_b1 = tf.constant(value = 0.0, shape = [n_hidden1])
#     b1 = tf.Variable(initial_value = initial_b1, name = 'b1')

#     layer1_in = tf.matmul(x, w1) + b1
#     layer1_out = tf.nn.relu(layer1_in)

### Layer 2

In [ ]:
# with tf.name_scope('second_layer'):

#     initial_w2 = tf.truncated_normal(shape = [n_hidden1, n_hidden2], stddev = 0.1, seed = 42)
#     w2 = tf.Variable(initial_value = initial_w2, name = 'w2')

#     initial_b2 = tf.constant(value = 0.0, shape = [n_hidden2])
#     b2 = tf.Variable(initial_value = initial_b2, name = 'b2')

#     layer2_in = tf.matmul(layer1_out, w2) + b2
#     layer2_out = tf.nn.relu(layer2_in)

### Layer 3

In [ ]:
# with tf.name_scope('output_layer'):

#     initial_w3 = tf.truncated_normal(shape = [n_hidden2, NR_CLASSES], stddev = 0.1, seed = 42)
#     w3 = tf.Variable(initial_value = initial_w3, name = 'w3')

#     initial_b3 = tf.constant(value = 0.0, shape = [NR_CLASSES])
#     b3 = tf.Variable(initial_value = initial_b3, name = 'b3')

#     layer3_in = tf.matmul(layer2_out, w3) + b3
#     output = tf.nn.softmax(layer3_in)

In [ ]:
def setup_layer(input, weight_dim, bias_dim, name):
    
    with tf.name_scope(name):

        initial_w = tf.truncated_normal(shape = weight_dim, stddev = 0.1, seed = 42)
        w = tf.Variable(initial_value = initial_w, name = 'W')

        initial_b = tf.constant(value = 0.0, shape = bias_dim)
        b = tf.Variable(initial_value = initial_b, name = 'B')

        layer_in = tf.matmul(input, w) + b
        
        if name == 'out':
            
            layer_out = tf.nn.softmax(layer_in)
            
        else:
            
            layer_out = tf.nn.relu(layer_in)
            
        tf.summary.histogram('weights', w)
        tf.summary.histogram('biases', b)
            
        return layer_out

In [ ]:
layer_1 = setup_layer(x, weight_dim = [TOTAL_INPUTS, n_hidden1], bias_dim = [n_hidden1], name = 'layer_1')

layer_drop = tf.nn.dropout(layer_1, keep_prob = 0.8, name = 'dropout_layer')

layer_2 = setup_layer(
    layer_drop, weight_dim = [n_hidden1, n_hidden2], 
    bias_dim = [n_hidden2], name = 'layer_2')

output = setup_layer(
    layer_2, weight_dim = [n_hidden2, NR_CLASSES], 
    bias_dim = [NR_CLASSES], name = 'out')

model_name = f'{n_hidden1}-DO-{n_hidden2} LR{learning_rate} E{nr_epochs}'

# Tensorboard setup

In [ ]:
folder_name = f'Model 1 at {strftime("%H:%M")}'
directory = os.path.join(LOGGING_PATH, folder_name)

try:
    os.makedirs(directory)
except OSError as exception:
    print(exception.strerror)
else:
    print('Successfully created directories!')

Successfully created directories!


# Loss, Optimization & Metrics

In [ ]:
with tf.name_scope('loss_calc'):
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = y, logits = output))


In [ ]:
with tf.name_scope('optimizer'):
    
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train_step = optimizer.minimize(loss)

## Accuracy Metric

In [ ]:
with tf.name_scope('accuracy_metric'):
    correct_pred = tf.equal(tf.argmax(output, axis = 1), tf.argmax(y, axis = 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [ ]:
with tf.name_scope('performance'):
    tf.summary.scalar('accuracy', accuracy)
    tf.summary.scalar('loss', loss)

### Check Input images in Tensorboard

In [ ]:
with tf.name_scope('show_image'): 
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('image_input', x_image, max_outputs=4)

# Run Session

In [ ]:
sess = tf.Session()

## Setup Filewrite and Merge Summaries

In [ ]:
merged_summary = tf.summary.merge_all()

train_writer = tf.summary.FileWriter(directory + '/train')
train_writer.add_graph(sess.graph)


validation_writer = tf.summary.FileWriter(directory + '/validation')

## Initialise all variables

In [ ]:
init = tf.global_variables_initializer()
sess.run(init)

In [ ]:
# b3.eval(sess)

## Batching the Data

In [ ]:
size_of_batch = 1000

num_examples = y_train.shape[0]
nr_iterations = int(num_examples/size_of_batch)

index_in_epoch = 0

In [ ]:
def next_batch(batch_size, data, labels):
    
    global num_examples
    global index_in_epoch
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    if index_in_epoch > num_examples:
        start = 0
        index_in_epoch = batch_size
        
    end = index_in_epoch
    
    return data[start:end], labels[start:end]

## Training loop

In [ ]:
for epoch in range(nr_epochs):
    
    for i in range(nr_iterations):
        
        batch_x, batch_y = next_batch(batch_size = size_of_batch, data = x_train, labels = y_train)
        
        feed_dictionary = {x:batch_x, y:batch_y}
        
        sess.run(train_step, feed_dict = feed_dictionary)
        
    
    s, batch_accuracy = sess.run(fetches = [merged_summary, accuracy], feed_dict = feed_dictionary)
        
    train_writer.add_summary(s, epoch)
    
    print(f'Epoch {epoch} \t| Training Accuracy = {batch_accuracy}')
    
    ## validation
    
    summary = sess.run(fetches=merged_summary, feed_dict = {x:x_validation, y:y_validation})
    validation_writer.add_summary(summary, epoch)

print('Done training.')

Epoch 0 	| Training Accuracy = 0.8489999771118164
Epoch 1 	| Training Accuracy = 0.8579999804496765
Epoch 2 	| Training Accuracy = 0.8659999966621399
Epoch 3 	| Training Accuracy = 0.8669999837875366
Epoch 4 	| Training Accuracy = 0.8690000176429749
Epoch 5 	| Training Accuracy = 0.8759999871253967
Epoch 6 	| Training Accuracy = 0.9769999980926514
Epoch 7 	| Training Accuracy = 0.9769999980926514
Epoch 8 	| Training Accuracy = 0.9779999852180481
Epoch 9 	| Training Accuracy = 0.9829999804496765
Done training.


# Make a Prediction

In [ ]:
img = Image.open('Handwriting/test_img.png')
img

In [ ]:
bw = img.convert('L')

In [ ]:
image_array = np.invert(bw)

In [ ]:
test_img = image_array.ravel()

In [ ]:
prediction = sess.run(fetches = tf.argmax(output, axis = 1), feed_dict = {x: [test_img]})
print(f'Prediction is: {prediction}')

Prediction is: [2]


In [ ]:
test_accuracy = sess.run(fetches = accuracy, feed_dict = {x:x_test, y:y_test})
print(f'Accuracy on test set is {test_accuracy:0.2%}')

Accuracy on test set is 96.41%


# Reset for next run

In [ ]:
train_writer.close()
sess.close()
tf.reset_default_graph()